In [15]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
import requests
from yelpapi import YelpAPI
import re
import time
import pickle

# Getting list of restaurant names from Yelp

In [37]:
yelp_api_key = 'f6lx4Zbdgaf6jPUk9g26HFx9lkuzXGTYWXiJibkLHEdbsqncM_sFy9dSBNshsVzabOYLEWaN8DogaeoNj_mOQHcdnlqsmPART6xhWxUvKOd_FkoCnskdIgP0ZZIXZXYx'
with YelpAPI(yelp_api_key) as yelp_api:
    response = yelp_api.search_query(term='chinese food', location='new york, ny')
    print(len(response))
    print(len(response['businesses']))

3
20


In [43]:
chinese_rests_yelp = []
for restaurant in response['businesses']:
    chinese_rests_yelp.append(restaurant['name'])
print(chinese_rests_yelp)

['Shu Jiao Fu Zhou', "Joe's Shanghai", 'Wah Fung No 1', 'Chow House', 'Blue Willow 夜来湘', 'The Rogue Panda', 'Zhongzhong Noodles - Brooklyn', 'Antidote', 'Noodle Village', 'Uncle Lou', 'Dim Sum Palace', 'Shanghai Villa', 'Lin & Daughters', 'Han Dynasty', 'Szechuan Mountain House', 'Birds of a Feather', 'Mama Lee - Williamsburg', "Bites of Xi'an", 'Baji Baji', 'New Chin Chin Wu Restaurant']


In [51]:
mandarin = chinese_rests_yelp[4]

In [62]:
english = re.sub(r'[^\x00-\x7F]+', '', mandarin)
english.strip()

'Blue Willow'

# Exploring Infatuation Page

### LOOK UP HOW TO GET MORE RESULTS IN A PAGE WITH LOAD MORE BUTTON

## Looping through Review Page URLs by page number 

In [16]:
review_urls = []
for i in range(1, 11):
    url = f"https://www.theinfatuation.com/new-york/reviews?page={i}"
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    review_urls += [link['href'] for link in soup.html.select('a') if "/reviews/" in link['href']]

In [7]:
len(review_urls)

255

In [17]:
review_urls

['/new-york/reviews/le-bernardin',
 '/new-york/reviews/thai-diner',
 '/new-york/reviews/lucali',
 '/new-york/reviews/estela',
 '/new-york/reviews/oxalis',
 '/new-york/reviews/taste-good-malaysian-cuisine',
 '/new-york/reviews/k-o-burger',
 '/new-york/reviews/clubhouse-sandos-and-subs',
 '/new-york/reviews/lupetto',
 '/new-york/reviews/downtown-burritos-cocina-mexicana',
 '/new-york/reviews/revelie-luncheonette',
 '/new-york/reviews/candlelight-inn-scarsdale',
 '/new-york/reviews/justines-on-hudson',
 '/new-york/reviews/metropolis-nyc',
 '/new-york/reviews/rockmeisha',
 '/new-york/reviews/kawabun-nyc',
 '/new-york/reviews/paradise-lost',
 '/new-york/reviews/roscioli-nyc',
 '/new-york/reviews/meju',
 '/new-york/reviews/sushi-counter',
 '/new-york/reviews/nine-cases',
 '/new-york/reviews/lindustrie-pizzeria-west-village',
 '/new-york/reviews/sailor',
 '/new-york/reviews/le-baratin-1',
 '/new-york/reviews/monsieur-vo',
 '/new-york/reviews/bread-and-butter',
 '/new-york/reviews/ayat-east-vi

In [20]:
resto_reviews = {}
url_prefix = "https://www.theinfatuation.com"
for count, url in enumerate(review_urls):
    review_page = urlopen(f"https://www.theinfatuation.com{url}")
    html = review_page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")

    # Get review dictionary from Soup
    review_dict = json.loads(soup.script.string)
    
    # Get resto name
    resto_name = review_dict['itemReviewed']['name']
    
    # Entire review body split into list containing normal review and food rundown section
    review_body = review_dict['reviewBody']
    review_split = review_body.split('Food Rundown')
    
    # Just review
    just_review = review_split[0]
    
    # Food Rundown
    if len(review_split) > 1:
        food_rundown = review_split[1]
    else:
        food_rundown = ''

    # Review price
    review_price = review_dict['itemReviewed']['priceRange']
    
    # Key words
    review_tags = review_dict['itemReviewed']['keywords']
    
    # Cuisine
    review_cuisine = review_dict['itemReviewed']['servesCuisine']
    
    # Date of Review
    review_date = review_dict['dateModified']
    
    resto_reviews[resto_name] = {
                                'review': just_review,
                                'food_rundown': food_rundown,
                                'cuisine': review_cuisine,
                                'perfect_for_tags': review_tags,
                                'price_range': review_price,
                                'review_date': review_date
                                }
    
    print(f"Got review for {resto_name}! Parsed {count + 1} restaurants so far.")

Got review for Le Bernardin! Parsed 1 restaurants so far.
Got review for Thai Diner! Parsed 2 restaurants so far.
Got review for Lucali! Parsed 3 restaurants so far.
Got review for Estela! Parsed 4 restaurants so far.
Got review for Oxalis! Parsed 5 restaurants so far.
Got review for Taste Good Malaysian Cuisine! Parsed 6 restaurants so far.
Got review for K.O. Burger! Parsed 7 restaurants so far.
Got review for Clubhouse Sandos &amp; Suds! Parsed 8 restaurants so far.
Got review for Lupetto! Parsed 9 restaurants so far.
Got review for Downtown Burritos Cocina Mexicana! Parsed 10 restaurants so far.
Got review for Revelie Luncheonette! Parsed 11 restaurants so far.
Got review for Candlelight Inn! Parsed 12 restaurants so far.
Got review for Justine’s On Hudson! Parsed 13 restaurants so far.
Got review for Metropolis! Parsed 14 restaurants so far.
Got review for Rockmeisha! Parsed 15 restaurants so far.
Got review for Kawabun! Parsed 16 restaurants so far.
Got review for Paradise Lost! 

Got review for Colonia Verde! Parsed 138 restaurants so far.
Got review for Fradei! Parsed 139 restaurants so far.
Got review for Olea! Parsed 140 restaurants so far.
Got review for Super Taste! Parsed 141 restaurants so far.
Got review for Saraghina Caffè! Parsed 142 restaurants so far.
Got review for Warude! Parsed 143 restaurants so far.
Got review for Foodstruck! Parsed 144 restaurants so far.
Got review for Ella Funt! Parsed 145 restaurants so far.
Got review for Nura! Parsed 146 restaurants so far.
Got review for OxKale! Parsed 147 restaurants so far.
Got review for Silver Apricot! Parsed 148 restaurants so far.
Got review for Sami &amp; Susu! Parsed 149 restaurants so far.
Got review for Scarr&apos;s Pizza! Parsed 150 restaurants so far.
Got review for L’Abeille à Côté! Parsed 151 restaurants so far.
Got review for Mogmog! Parsed 152 restaurants so far.
Got review for Sami&apos;s Kabab House LIC! Parsed 153 restaurants so far.
Got review for R40! Parsed 154 restaurants so far.
G

In [10]:
resto_reviews['Monsieur Vo']

{'review': 'When we dine out at Vietnamese restaurants, there’s one question we often ask ourselves. Is it okay to order a whole sandwich alongside an entire meal? The banh mi board at Monsieur Vo, bougie as it sounds, is a genius solution to this dilemma, with the sandwich components laid out like a charcuterie plate. That’s only one of the hits at this lively East Village restaurant with flowery, dark blue wallpaper, arches, and warm, welcoming lighting. This is good choice for a casual date night, or a catch-up with friends—get the fried okra and tamarind ribs, and, if you’re sharing with a few people, order the impressive beef shank or whole fried sea bass. ',
 'price_range': '$$',
 'perfect_for_tags': 'Vietnamese'}

In [21]:
# Write infatuation restaurant reviews to pkl file
with open('infatuation_reviews_v2.pkl', 'wb') as file:
    pickle.dump(resto_reviews, file, protocol=pickle.HIGHEST_PROTOCOL)

# Read infatuation restaurant reviews from pkl file
# with open('infatuation_reviews.pkl', 'rb') as file:
#     loaded_resto_reviews = pickle.load(file)

### Page by Page of review page

In [2]:
# Get Soup from URL
url = "https://www.theinfatuation.com/new-york/reviews"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [27]:
url2 = "https://www.theinfatuation.com/new-york/reviews?page=2"
page2 = urlopen(url2)
html2 = page2.read().decode("utf-8")
soup2 = BeautifulSoup(html2, "html.parser")

In [28]:
soup2.html

<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="The Infatuation" property="og:site_name"/><meta content="max-image-preview:large" name="robots"/><meta content="https://www.theinfatuation.com/new-york/reviews?page=2" name="parsely-link"/><meta content="index" name="parsely-type"/><meta content="reviews?page=2" name="parsely-tags"/><meta content="new-york" name="parsely-section"/><title>New York Restaurant Reviews - Page 2 - The Infatuation</title><meta content="New York Restaurant Reviews - Page 2 - The Infatuation" property="og:title"/><meta content="New York Restaurant Reviews - Page 2 - The Infatuation" name="parsely-title"/><link href="https://www.theinfatuation.com/new-york/reviews?page=2" rel="canonical"/><meta content="12" name="next-head-count"/><meta name="app_version"/><link href="/assets/favicon_48.png" rel="icon"/><link href="/assets/favicon_16.png" rel="icon" sizes="16x16" type="image/png"/><

In [29]:
len(soup2.html.select('a'))

245

In [31]:
soup2.html.select('a')[10]['href']

'/new-york/cuisines/salads'

In [32]:
soup2.html.select('a')[0]['href']

'/'

In [16]:
review_urls = [link['href'] for link in soup2.html.select('a') if "/reviews/" in link['href']]

In [17]:
len(review_urls)

30

In [12]:
url = "https://www.theinfatuation.com/new-york/reviews/steak-frites"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [13]:
review_dict = json.loads(soup.script.string)
# Review body
review_body = review_dict['reviewBody']

In [14]:
review_body

'Naming a place after a dish may seem lazy, but it’s actually bold. You have to be able to back it up, and Steak Frites does, serving textbook interpretations of classic bistro food—including a flawless version of its namesake dish. If you’re looking for something  smothered in foam under a smoke-filled cloche, go elsewhere. If you’re looking for straightforward, satisfying French fare, go to Steak Frites.\xa0 Appropriately, this Hell’s Kitchen restaurant looks like the end result of a director telling a set designer: “Build me a neighborhood Parisian bistro where we can film our meet-cute.” Worn wood furniture, distressed gold-framed mirrors, and vintage cartoon postcards make this place feel decades old. It’s bustling, loud, and cramped, with waiters constantly bumping into each other, and glasses getting knocked over about once every hour. If you come often enough, you’ll eventually break something too, unless you score a seat at the less chaotic bar. Ordering here involves no major

## Infatuation Search Query URL

In [31]:
# Get Soup from URL
url = "https://www.theinfatuation.com/finder?query=chinese&location=New+York&canonicalPath=%2Fnew-york&postType=UNSPECIFIED"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [32]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="The Infatuation" property="og:site_name"/><meta content="max-image-preview:large" name="robots"/><meta content="https://www.theinfatuation.com/finder" name="parsely-link"/><meta content="index" name="parsely-type"/><meta content="webpage" name="parsely-tags"/><meta content="finder" name="parsely-section"/><title>Find Restaurants - The Infatuation</title><meta content="Find Restaurants - The Infatuation" property="og:title"/><meta content="Find Restaurants - The Infatuation" name="parsely-title"/><meta content="11" name="next-head-count"/><meta name="app_version"/><link href="/assets/favicon_48.png" rel="icon"/><link href="/assets/favicon_16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/assets/favicon_32.png" rel="icon" sizes="32x32" type="image/png"/><link href="/assets/favicon_48.png" rel="icon" sizes="48x48" type="image/png"

## Szechuan Mountain House

In [39]:
# Get Soup from URL
url = "https://www.theinfatuation.com/new-york/reviews/szechuan-mountain-house"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

# Get review dictionary from Soup
review_dict = json.loads(soup.script.string)

# Review body
review_body = review_dict['reviewBody']

# Review price
review_price = review_dict['itemReviewed']['priceRange']

In [40]:
review_dict

{'@context': 'https://schema.org',
 '@type': 'Review',
 'author': [{'@type': 'Person',
   'image': 'https://res.cloudinary.com/the-infatuation/image/upload/q_auto,f_auto/cms/hannah_202',
   'name': 'Hannah Albertine',
   'sameAs': 'https://www.theinfatuation.com/contributor/hannah-albertine'}],
 'datePublished': '2022-01-21T04:56:36.444Z',
 'dateModified': '2022-08-22T23:43:13.801Z',
 'description': 'Szechuan Mountain House is a fun Szechuan restaurant to bring a date or a friend on St. Marks Place in the East Village.',
 'inLanguage': 'en',
 'itemReviewed': {'@type': 'FoodEstablishment',
  'address': {'@type': 'PostalAddress',
   'addressCountry': '',
   'addressLocality': 'New York',
   'addressRegion': 'NY',
   'name': '23 Saint Marks Pl, New York, NY 10003',
   'postalCode': '10003',
   'streetAddress': '23 Saint Marks Pl'},
  'geo': {'@type': 'GeoCoordinates',
   'latitude': 40.7291232028,
   'longitude': -73.9886718492},
  'image': 'https://res.cloudinary.com/the-infatuation/imag

In [7]:
review_body

'If you’re looking for great food on St. Marks Place between 2nd and 3rd Avenues, you’re probably also looking for a good time (and possibly a lot of alcohol, then a second cartilage piercing). Uplifting news for you: Szechuan Mountain House exists. The Szechuan food here is memorably numbing and garlic-heavy. If you’re interested in outdoor dining, there’s a patio area out front that’s decorated with plants and dividers where you can hang out and drink a few bottles of Tsing Tao to offset the mala spice.  No matter what you order, your meal starts here with complimentary homemade pickled cabbage and ends with cold mung bean soup. In between, we’d suggest getting the mapo tofu, which is both stomach-vibrating and delicious, as well as the pork belly with garlic. It’s possible you’ve seen pictures of it on social media: it comes with cucumbers and light-pink, thinly sliced pork slabs hanging neatly over a rack that looks like something you’d use to dry jeans. Combined with a helping of 

## CheLi

In [4]:
# Get Soup from URL
url = "https://www.theinfatuation.com/new-york/reviews/cheli"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

# Get review dictionary from Soup
review_dict = json.loads(soup.script.string)

# Review body
review_body = review_dict['reviewBody']

# Review price
review_price = review_dict['itemReviewed']['priceRange']

In [5]:
review_body

'Take a moment to picture the anarchic realm of St. Marks between 2nd and 3rd Avenues. See the puffing clouds of blueberry vapes, the exodus of NYU juniors toting packs of pore-minimizing face masks, and the folding chair in front of Mamoun’s where a man dressed like the year 1983 asks if you’re interested in adding CBD to your falafel.  Now, imagine the opposite: the peaceful chortle of a waterfall, friends clinking Tsingtao bottles under indoor thatched roofs, wafts of beautifully smoked fish and braised pork slick and glossy in a caramelized soy sauce. CheLi, a serene, banquet-style Shanghainese restaurant, exists just a staircase away from the belly of St. Marks, but you’d never know it once you’re inside. New York has plenty of places to eat Shanghainese food with a group (you could walk 50 steps from CheLi and ride off into the sunset with a bunch of xiao long bao at The Bao). Even still, we can’t think of many restaurants overtly spotlighting food from the Jiangnan region surrou

## Nowon

In [14]:
# Get Soup from URL
url = "https://www.theinfatuation.com/new-york/reviews/nowon"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

# Get review dictionary from Soup
review_dict = json.loads(soup.script.string)

# Review body
review_body = review_dict['reviewBody']

# Key words
review_tags = review_dict['itemReviewed']['keywords']

# Review price
review_price = review_dict['itemReviewed']['priceRange']

In [70]:
review_dict['itemReviewed']['name']

'Nowon'

In [9]:
review_body

'Nowon is a new Korean spot in the East Village that has a double-patty steak burger worth traveling for. The burger originally gained a following through a pop-up at Black Emperor - but now it has its own home here. This dimly-lit restaurant doesn’t serve alcohol yet, but they&apos;re always playing loud &apos;90s hip-hop and you can expect some Korean iced teas, bar snacks like honey butter tater tots.'

In [15]:
review_tags

'Casual Weeknight Dinner, Dining Solo, Eating At The Bar, First/Early in the Game Dates, Korean'

In [10]:
review_price

'$$'

## Looping through Yelp list of restaurants and querying from Infatuation URL

In [67]:
created_infatuation_urls = []
for chinese_resto in chinese_rests_yelp:
    chinese_resto = re.sub(r'[^\x00-\x7F]+', '', chinese_resto)
    chinese_resto = chinese_resto.strip()
    chinese_resto = chinese_resto.replace(" ", "-").replace("'", "").lower()
#     print(chinese_resto)
    created_infatuation_urls.append(f"https://www.theinfatuation.com/new-york/reviews/{chinese_resto}")

In [73]:
created_infatuation_urls

['https://www.theinfatuation.com/new-york/reviews/shu-jiao-fu-zhou',
 'https://www.theinfatuation.com/new-york/reviews/joes-shanghai',
 'https://www.theinfatuation.com/new-york/reviews/wah-fung-no-1',
 'https://www.theinfatuation.com/new-york/reviews/chow-house',
 'https://www.theinfatuation.com/new-york/reviews/blue-willow',
 'https://www.theinfatuation.com/new-york/reviews/the-rogue-panda',
 'https://www.theinfatuation.com/new-york/reviews/zhongzhong-noodles---brooklyn',
 'https://www.theinfatuation.com/new-york/reviews/antidote',
 'https://www.theinfatuation.com/new-york/reviews/noodle-village',
 'https://www.theinfatuation.com/new-york/reviews/uncle-lou',
 'https://www.theinfatuation.com/new-york/reviews/dim-sum-palace',
 'https://www.theinfatuation.com/new-york/reviews/shanghai-villa',
 'https://www.theinfatuation.com/new-york/reviews/lin-&-daughters',
 'https://www.theinfatuation.com/new-york/reviews/han-dynasty',
 'https://www.theinfatuation.com/new-york/reviews/szechuan-mountai

In [79]:
resto_reviews = {}
for count, url in enumerate(created_infatuation_urls):
    # Get Soup from URL
    try:
        page = urlopen(url)
    except Exception as e:
        print(f"Exception {e} when hitting URL: {url}!")
        continue
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    # Get review dictionary from Soup
    review_dict = json.loads(soup.script.string)
    # Restaurant Name
    resto_name = chinese_rests_yelp[count]
    # Review body
    review_body = review_dict['reviewBody']
    # Key words
    review_tags = review_dict['itemReviewed']['keywords']
    # Review price
    review_price = review_dict['itemReviewed']['priceRange']
    
    resto_reviews[resto_name] = {'review': review_body, 
                                      'perfect_for_tags': review_tags, 
                                      'price_range': review_price}
    
    print(f"Got review for {resto_name}! Parsed {count + 1} restaurants so far.")
    
    if count < (len(created_infatuation_urls) - 1):
        time.sleep(10)
    

Exception HTTP Error 404: Not Found when hitting URL: https://www.theinfatuation.com/new-york/reviews/shu-jiao-fu-zhou!
Got review for Joe's Shanghai! Parsed 2 restaurants so far.
Exception HTTP Error 404: Not Found when hitting URL: https://www.theinfatuation.com/new-york/reviews/wah-fung-no-1!
Exception HTTP Error 404: Not Found when hitting URL: https://www.theinfatuation.com/new-york/reviews/chow-house!
Got review for Blue Willow 夜来湘! Parsed 5 restaurants so far.
Exception HTTP Error 404: Not Found when hitting URL: https://www.theinfatuation.com/new-york/reviews/the-rogue-panda!
Exception HTTP Error 404: Not Found when hitting URL: https://www.theinfatuation.com/new-york/reviews/zhongzhong-noodles---brooklyn!
Got review for Antidote! Parsed 8 restaurants so far.
Got review for Noodle Village! Parsed 9 restaurants so far.
Exception HTTP Error 404: Not Found when hitting URL: https://www.theinfatuation.com/new-york/reviews/uncle-lou!
Exception HTTP Error 404: Not Found when hitting 

In [88]:
for resto in resto_reviews:
    print(f"{resto}:")
    print(f"\nReview: {resto_reviews[resto]['review']}\n\nKey Words: {resto_reviews[resto]['perfect_for_tags']}\n\nPrice Range: {resto_reviews[resto]['price_range']}\n\n----------------------------------------------------------------\n\n")

Joe's Shanghai:

Review: Joe Shanghai is often the first spot we take friends, family, or anybody else who’s in town and requests a dinner out in Chinatown. Their dining room is always lively, and the soup dumplings are on every table. They have two options on the menu (crab with pork and just pork), and you should absolutely get at least one order of each. The wrappers are delicate, there’s a perfect ratio of meat to soup, and they’re always piping hot. Depending on the size of your group, you could come here for a couple orders of soup dumplings and be in and out in under 30 minutes. But we think you should stay a little longer for the griddled rice cakes with shredded cabbage and pork and some garlicky green beans with minced pork.

Key Words: Classic Establishment, Chinese

Price Range: $$

----------------------------------------------------------------


Blue Willow 夜来湘:

Review: If you’re near the bottom of Central Park and looking for great Chinese food, check out this Szechuan